In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

driver = webdriver.Chrome() 
driver.get("https://jobsearchmalawi.com/")

def scrape_listings(page_source):
    soup = BeautifulSoup(page_source, 'html.parser')
    job_listings = soup.find_all('li', class_='job_listing')
    listings_data = []
    for listing in job_listings:
        job_position_elem = listing.find('h3')
        job_position = job_position_elem.text.strip() if job_position_elem else None
        if job_position == "How to Search For a Job Advert":  
            continue  

        company_elem = listing.find('div', class_='company')
        company = company_elem.strong.text.strip() if company_elem and company_elem.strong else None
        if company is None:
            
            continue

        location_elem = listing.find('div', class_='location')
        location = location_elem.text.strip() if location_elem else None
        if location is None:
            
            continue

        job_type_elem = listing.find('li', class_='job-type')
        job_type = job_type_elem.text.strip() if job_type_elem else None
        if job_type is None:
            
            continue

        posting_date_elem = listing.find('li', class_='date')
        posting_date = posting_date_elem.time['datetime'] if posting_date_elem else None
        if posting_date is None:
            
            continue

        listing_data = {
            'Position': job_position,
            'Company': company,
            'Location': location,
            'Job Type': job_type,
            'Posting Date': posting_date
        }
        listings_data.append(listing_data)
    return listings_data

new_listings = []

while True:
    try:
        load_more_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "load_more_jobs"))
        )
        load_more_button.click()
        time.sleep(2)  
        page_source = driver.page_source
        scraped_listings = scrape_listings(page_source)
        if not scraped_listings:
            break
        new_listings.extend(scraped_listings)
    except Exception as e:
        
        break  

driver.quit()

if new_listings:
    df = pd.DataFrame(new_listings)
   
else:
    print("No new listings found.")

df.to_csv('jobsearch_listings.csv', index=False)


There was an error managing chromedriver (error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json): error trying to connect: dns error: No such host is known. (os error 11001)); using driver found in the cache


No new listings found.


NameError: name 'df' is not defined